In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time 
import datetime
device = torch.device('cuda' if torch.backends.cuda.is_built() else 'cpu')
def format_time(time):
    elapsed_rounded = int(round((time)))
    # 格式化为 hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
 

# 设置随机种子
torch.manual_seed(42)

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载MNIST数据集
train_dataset = datasets.MNIST('./data', train=True, download=False, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=False, transform=transform)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义卷积神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(1600,9216)
        self.fc2 = nn.Linear(9216, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        return x

# 创建模型实例
model = Net().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print('[Epoch %d, Batch %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

# 测试模型
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test accuracy: %.2f%%' % (100 * correct / total))

# 训练和测试模型

begin = time.time()
train(model, train_loader, criterion, optimizer, epochs=10)
end = time.time()
training_time = format_time(end-begin)
print(f'训练时间为：{training_time}')
# time consuming
test(model, test_loader)

[Epoch 1, Batch   100] loss: 0.448
[Epoch 1, Batch   200] loss: 0.110
[Epoch 1, Batch   300] loss: 0.107
[Epoch 1, Batch   400] loss: 0.087
[Epoch 1, Batch   500] loss: 0.060
[Epoch 1, Batch   600] loss: 0.079
[Epoch 1, Batch   700] loss: 0.063
[Epoch 1, Batch   800] loss: 0.061
[Epoch 1, Batch   900] loss: 0.052
[Epoch 2, Batch   100] loss: 0.040
[Epoch 2, Batch   200] loss: 0.042
[Epoch 2, Batch   300] loss: 0.035
[Epoch 2, Batch   400] loss: 0.038
[Epoch 2, Batch   500] loss: 0.046
[Epoch 2, Batch   600] loss: 0.041
[Epoch 2, Batch   700] loss: 0.042
[Epoch 2, Batch   800] loss: 0.046
[Epoch 2, Batch   900] loss: 0.038
[Epoch 3, Batch   100] loss: 0.023
[Epoch 3, Batch   200] loss: 0.015
[Epoch 3, Batch   300] loss: 0.034
[Epoch 3, Batch   400] loss: 0.025
[Epoch 3, Batch   500] loss: 0.029
[Epoch 3, Batch   600] loss: 0.025
[Epoch 3, Batch   700] loss: 0.030
[Epoch 3, Batch   800] loss: 0.031
[Epoch 3, Batch   900] loss: 0.033
[Epoch 4, Batch   100] loss: 0.015
[Epoch 4, Batch   20